## Extract footprint information from long time-series s3l2 products.
#### 1. The nearest valid footprint and the corresponding informatio, e.g.,waveform.
#### 2. The ransac prediction ssha value corresponding to the nearest footprint (may be not the valid ssha).
#### 3. Interpolation result with two time-nearest tgauge data.
#### 4. ssha difference of two time-rearest tgauge data.


In [12]:
import matplotlib.pyplot as plt
from get_filesPath import get_filesPath
from s3l2_extract import s3l2_extract
from tgauge_data_interp import tgauge_data_interp
import sys
sys.path.append("../utils")
import xarray as xr
import numpy as np
import os
from tiff_io import readTiff
from imgShow import imgShow


In [13]:
root_dir = os.path.dirname(os.getcwd())
dir_s3l2 = root_dir+'/test_data/s3a-relative-orbit-260&309'
dir_output = root_dir + '/results'


In [14]:
## data prepare
# 1. get files path
files_path = get_filesPath(base=dir_s3l2, key_words='enhanced_measurement_subs_preproc_mainproc_postproc.nc')
files_path = sorted(files_path)
print('s3l2 files number:', len(files_path))


s3l2 files number: 50


In [8]:
retrackers = ['ocean', 'ocog', 'ice_sheet', 'sea_ice']
for retracker in retrackers:
    ''' 1. the valid nearest footprint are vary with retrackers, thus the features (e.g., waveform) to be added to the .nc file is different with retrackers.
        2. the nearest valid footprint position may be different, but the nearest footprint position is the same 
    '''
    print('retracker:', retracker)
    for i, path_s3l2 in enumerate(files_path):
        s3_l2 = xr.open_dataset(path_s3l2)
        ## determine indexes of the nearest footprint and valid nearest footprint.
        ind_fp_near = np.argmin(s3_l2['dis_fp_tgauge'].values)
        ind_nan = np.isnan(s3_l2['ssha_land_wf_ransac_filter_'+retracker])
        s3_l2['dis_fp_tgauge'][ind_nan] = np.nan
        ind_fp_near_valid = np.nanargmin(s3_l2['dis_fp_tgauge'])
        
        ## 1. extract footprint information from s3l2, e.g., time, waveform,location...
        fp_record = s3l2_extract(s3_l2, ind_fp_near_valid)
        
        ## 2. add nearest ransac prediction variable
        ## 2.1 land randsac prediction
        fp_record['ssha_land_ransac_pred_'+retracker] = s3_l2['ssha_land_ransac_pred_'+retracker].values[ind_fp_near]
        fp_record['ssha_land_ransac_pred_'+retracker].attrs['name'] = 'land filtered ransac prediction corresponding to the nearest footprint to tgauge station'
        ## 2.2 wf randsac prediction
        fp_record['ssha_wf_ransac_pred_'+retracker] = s3_l2['ssha_wf_ransac_pred_'+retracker].values[ind_fp_near]
        fp_record['ssha_wf_ransac_pred_'+retracker].attrs['name'] = 'wf filtered ransac prediction corresponding to the nearest footprint to tgauge station'
        # 2.3 land+wf filtered ransac prediction
        fp_record['ssha_land_wf_ransac_pred_'+retracker] = s3_l2['ssha_land_wf_ransac_pred_'+retracker].values[ind_fp_near]
        fp_record['ssha_land_wf_ransac_pred_'+retracker].attrs['name'] = 'land and wf filtered ransac prediction corresponding to the nearest footprint to tgauge station'

        ## 3. add tgauge station location
        fp_record['lon_tgauge'] = s3_l2['lon_tgauge'].values
        fp_record['lat_tgauge'] = s3_l2['lat_tgauge'].values
        
        ## 4.linear interpolation of the tgauge data
        target_time = s3_l2['time_20_ku'][ind_fp_near_valid]
        ssha_tgauge_interp, ind_time_start = tgauge_data_interp(tgauge_values=s3_l2['ssha_tgauge'], tgauge_times=s3_l2['time_tgauge'], target_time= target_time)
        ssha_tgauge_interp = ssha_tgauge_interp-1.38 ## specify to Hong Kong region
        fp_record['ssha_tgauge_interp'] = ssha_tgauge_interp.values
        fp_record['ssha_tgauge_interp'].attrs['unit'] = 'meter'
        
        ## 5. add ssha difference between two nearest times of tgauge data
        fp_record['tgauge_ssha_dif_near2'] = abs(s3_l2['ssha_tgauge'][ind_time_start]-s3_l2['ssha_tgauge'][ind_time_start+1])
        fp_record['tgauge_ssha_dif_near2'].attrs['name'] = 'ssha difference between two neareast tgauge time to the satellite data acquisition time'
        fp_record['tgauge_ssha_dif_near2'].attrs['unit'] = 'meter'
        ## concatenation of the obtained data
        if i == 0:
            fp_records = fp_record
            continue
        fp_records = xr.concat([fp_records, fp_record], dim='time_20_ku', coords='all')
        print('time:', fp_record['time_20_ku'].values, end=' ')
        print('ssha_tgauge: %.3f' % fp_record['ssha_tgauge_interp'].values, end=' ')
        print('ssha_estimate: %.3f' % fp_record['ssha_estimate_'+retracker].values)
    ## writh to new .nc file
    path_result = os.path.join(dir_output, 'result_final_'+retracker+'.nc')
    fp_records.to_netcdf(path=path_result)


retracker: ocean
time: 2019-01-24T14:08:49.746136704 ssha_tgauge: 1.016 ssha_estimate: 0.854
time: 2019-02-17T02:37:22.816082432 ssha_tgauge: 0.016 ssha_estimate: -0.003
time: 2019-02-20T14:08:50.092905984 ssha_tgauge: 0.951 ssha_estimate: 0.975
time: 2019-03-16T02:37:19.856276096 ssha_tgauge: 0.368 ssha_estimate: -0.036
time: 2019-03-19T14:08:48.966854272 ssha_tgauge: 0.238 ssha_estimate: 0.629
time: 2019-04-12T02:37:25.660643840 ssha_tgauge: 0.680 ssha_estimate: 0.560
time: 2019-04-15T14:08:53.921298560 ssha_tgauge: -0.393 ssha_estimate: -0.314
time: 2019-05-09T02:37:28.304749696 ssha_tgauge: 1.045 ssha_estimate: 1.046
time: 2019-05-12T14:08:56.330205440 ssha_tgauge: -0.737 ssha_estimate: -0.754
time: 2019-06-05T02:37:27.693831168 ssha_tgauge: 0.749 ssha_estimate: 0.673
time: 2019-06-08T14:08:55.368344832 ssha_tgauge: -0.566 ssha_estimate: -0.886
time: 2019-07-02T02:37:27.366581248 ssha_tgauge: 0.390 ssha_estimate: 0.633
time: 2019-07-05T14:08:55.213489536 ssha_tgauge: -0.214 ssha_es

In [9]:
# s3_l2
# fp_records
